In [8]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import shutil
import image_utils

from sklearn.metrics.pairwise import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import torch
import torchvision
import torch.nn as nn
from torchvision import transforms


%matplotlib inline

# Load Data

In [2]:
data_path = './test_images/dresses_sample/*.jpg'

In [3]:
# read data
filelist = glob.glob(data_path)
filelist = sorted(filelist)
print('num images: ', len(filelist))

num images:  500


In [4]:
filenames = [os.path.basename(f).split('.jpg')[0] for f in filelist]
filenames = sorted(filenames)

# Get Features From Retrained ResNet152

In [ ]:
feature_path = './test_features/dresses_sample/resnet_2048/'

In [ ]:
model_path = "./transfer_learning/best_model.pth"
model = torchvision.models.resnet152(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)
model.load_state_dict(torch.load(model_path, map_location='cpu'))
modules = list(model.children())[:-1]
model_last = nn.Sequential(*modules)

In [ ]:
data_transforms = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
image_utils.save_feature_vectors_from_model(filelist, feature_path, model_last, data_transforms)

# Get Features From Original ResNet152

In [ ]:
model = torchvision.models.resnet152(pretrained=True)
modules = list(model.children())[:-1]
model_last = nn.Sequential(*modules)

In [9]:
feature_path = './test_features/dresses_sample/resnet_orig_2048/'
image_utils.save_feature_vectors_from_model(filelist, feature_path, model_last, data_transforms)

# Get PCA From Original ResNet152

In [31]:
resnet_orig_feats = np.array([np.load(f) for f in glob.glob(feature_path + '*.npy')])
resnet_orig_feats.shape

(500, 2048)

In [32]:
scaler = StandardScaler()
resnet_feats_std = scaler.fit_transform(resnet_orig_feats)

In [33]:
sklearn_pca = PCA(n_components=128)
resnet_feats_pca = sklearn_pca.fit_transform(resnet_feats_std)
resnet_feats_pca.shape

(500, 128)

In [34]:
pca_feature_path = './test_features/dresses_sample/resnet_pca_128/'
if not os.path.exists(pca_feature_path):
    os.makedirs(pca_feature_path)

In [35]:
for idx, filename in enumerate(filenames):
    np.save(os.path.join(pca_feature_path, filename + '.npy'), resnet_feats_pca[idx])

# Get Features From Akiwi

In [6]:
feature_path = './test_features/dresses_sample/akiwi_114/'
image_utils.download_feature_vectors_114(filelist, feature_path)

Downloaded 0 / 500
Downloaded 100 / 500
Downloaded 200 / 500
Downloaded 300 / 500
Downloaded 400 / 500
